# Setup

In [6]:
%cd /kaggle/working/
%rm -rf mlproduction
!git clone https://github.com/daoanhkhoa123/mlproduction.git
%cd /kaggle/working/mlproduction
!ls

/kaggle/working
Cloning into 'mlproduction'...
remote: Enumerating objects: 353, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 353 (delta 13), reused 29 (delta 10), pack-reused 317 (from 1)
Receiving objects: 100% (353/353), 3.58 MiB | 14.03 MiB/s, done.
Resolving deltas: 100% (201/201), done.
/kaggle/working/mlproduction
main.py  pyproject.toml  README.md  runtime  scripts  sqls  src  uv.lock


In [7]:
!chmod +x scripts/*.sh
!scripts/cloud_setup.sh

Resolved 69 packages in 378ms                                        
# This file was autogenerated by uv via the following command:
#    uv pip compile pyproject.toml -o requirements.txt --extra-index-url https://pypi.org/simple --emit-index-url
--index-url https://pypi.org/simple
--extra-index-url https://pypi.org/simple

aiohappyeyeballs==2.6.1
    # via aiohttp
aiohttp==3.13.2
    # via fsspec
aiosignal==1.4.0
    # via aiohttp
annotated-types==0.7.0
    # via pydantic
attrs==25.4.0
    # via aiohttp
certifi==2025.11.12
    # via requests
charset-normalizer==3.4.4
    # via requests
filelock==3.20.1
    # via
    #   huggingface-hub
    #   torch
    #   transformers
frozenlist==1.8.0
    # via
    #   aiohttp
    #   aiosignal
fsspec==2025.12.0
    # via
    #   huggingface-hub
    #   lightning
    #   pytorch-lightning
    #   torch
hf-xet==1.2.0
    # via huggingface-hub
huggingface-hub==0.36.0
    # via
    #   tokenizers
    #   transformers
idna==3.11
    # via
    #   reque

# Run

In [8]:
from runtime.ultils import gdown_folder
gdown_folder()

Retrieving folder contents


Retrieving folder 1aoddGUNcsdJuD6KtSRigVnqLW4uqeHin .ipynb_checkpoints
Processing file 1pQevQ6Ors5DVuAXLxXiWkGh9LGi63DjE hallu_classes.csv
Processing file 1BWfYcKTw1lBcEpeINvcsL2sW36hsadYT hallu_data.csv


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1pQevQ6Ors5DVuAXLxXiWkGh9LGi63DjE
To: /kaggle/working/mlproduction/data/hallu_classes.csv
100%|██████████| 25.0/25.0 [00:00<00:00, 78.1kB/s]
Downloading...
From: https://drive.google.com/uc?id=1BWfYcKTw1lBcEpeINvcsL2sW36hsadYT
To: /kaggle/working/mlproduction/data/hallu_data.csv
 44%|████▍     | 4.72M/10.7M [00:00<00:00, 27.5MB/s]

Folder downloaded successfully.


100%|██████████| 10.7M/10.7M [00:00<00:00, 52.5MB/s]
Download completed


# What is the dog doing?


In [9]:
import dagshub
dagshub.init(repo_owner='ollamagena', repo_name='mlpro', mlflow=True)


Repository mlpro doesn't exist, creating it under current user.

Initialized MLflow to track repo "ollamagena/mlpro"

Repository ollamagena/mlpro initialized!

# Run

In [10]:
import pandas as pd
from src.dataloaders.ultils import train_test_split_df, load_label_enocder

df = pd.read_csv("data/hallu_data.csv")
df.head()
le = load_label_enocder("data/hallu_classes.csv")
X_train, X_test, y_train, y_test = train_test_split_df(df, "label", le, 0.3)
len(X_train)

ImportError: cannot import name 'train_test_split_df' from 'src.dataloaders.ultils' (/kaggle/working/mlproduction/src/dataloaders/ultils.py)

In [ ]:
from src.models.preprop_svm import Preprop, prep_remove_common_words_with_counting, agg_differential
from sklearn.svm import SVC

preprop = Preprop(prep_remove_common_words_with_counting, agg_differential)
svc = SVC()